In [4]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:8 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [736 kB]
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,564 kB]
Get:13 http://archive.ubuntu.com/ubuntu

In [5]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [6]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
home_sales_df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [7]:
# create a temporary view of the DataFrame
home_sales_df.createOrReplaceTempView('home_sales')

In [8]:
# What is the average price for a four bedroom house sold per year
sql = """
SELECT YEAR(date) AS YEAR, ROUND(AVG(price), 2) AS AVG_PRICE
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR
ORDER BY YEAR
"""

spark.sql(sql).show()

+----+---------+
|YEAR|AVG_PRICE|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [9]:
# What is the average price of a home each year it was built,
# that have 3 bedrooms and 3 baths rounded to two decimals
sql = """
SELECT YEAR(date) AS YEAR, ROUND(AVG(price), 2) AS AVG_PRICE
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY YEAR
ORDER BY YEAR
"""

spark.sql(sql).show()

+----+---------+
|YEAR|AVG_PRICE|
+----+---------+
|2019|287287.82|
|2020|294204.16|
|2021|294211.46|
|2022|292725.69|
+----+---------+



In [11]:
# What is the average price of a home each year it was built,
# that have 3 bedrooms and 3 baths with two floors,
# and are >= to 2000 sqft roudned 2 decimal places
sql = """
SELECT YEAR(date_built) AS YEAR, ROUND(AVG(price), 2) AS AVG_PRICE
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 20000
GROUP BY YEAR
ORDER BY YEAR
"""

spark.sql(sql).show()

+----+---------+
|YEAR|AVG_PRICE|
+----+---------+
+----+---------+



In [13]:
# What is the average price of a home per "view" rating having an average home price greater than or equal to $350,000
# Determine the run time for this query, and round off your answer to two decimal places.

start_time = time.time()

sql = """
SELECT view, ROUND(AVG(price), 2) AS AVG_PRICE
FROM home_sales
GROUP by view
HAVING AVG_PRICE >= 350000
ORDER BY view
"""

spark.sql(sql).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|AVG_PRICE|
+----+---------+
| 100|1026669.5|
|  51|788128.21|
|  52|733780.26|
|  53| 755214.8|
|  54|798684.82|
|  55|771153.32|
|  56| 718176.4|
|  57| 734340.5|
|  58|759764.65|
|  59| 791453.0|
|  60|754939.65|
|  61|746877.59|
|  62|759150.14|
|  63|711614.55|
|  64|767036.67|
|  65|736679.93|
|  66| 712475.0|
|  67|737970.96|
|  68|716785.44|
|  69|750537.94|
+----+---------+
only showing top 20 rows

--- 0.770028829574585 seconds ---


In [15]:
# cache the temporary table home_sales
spark.sql('cache table home_sales')


DataFrame[]

In [16]:
# check if the table is cached
spark.catalog.isCached('home_sales')

True

In [18]:
# run the last query with cached data table
# What is the average price of a home per "view" rating having an average home price greater than or equal to $350,000
# Determine the run time for this query, and round off your answer to two decimal places.

start_time = time.time()

sql = """
SELECT view, ROUND(AVG(price), 2) AS AVG_PRICE
FROM home_sales
GROUP by view
HAVING AVG_PRICE >= 350000
ORDER BY view
"""

spark.sql(sql).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|AVG_PRICE|
+----+---------+
| 100|1026669.5|
|  51|788128.21|
|  52|733780.26|
|  53| 755214.8|
|  54|798684.82|
|  55|771153.32|
|  56| 718176.4|
|  57| 734340.5|
|  58|759764.65|
|  59| 791453.0|
|  60|754939.65|
|  61|746877.59|
|  62|759150.14|
|  63|711614.55|
|  64|767036.67|
|  65|736679.93|
|  66| 712475.0|
|  67|737970.96|
|  68|716785.44|
|  69|750537.94|
+----+---------+
only showing top 20 rows

--- 0.808769702911377 seconds ---


In [19]:
# Partition the home sales dataset by date_built field
home_sales_df.write.partitionBy('date_built').parquet('p_home_sales', mode ='overwrite')

In [21]:
# Read parquet data
p_home_sales_df = spark.read.parquet('p_home_sales')

In [22]:
# Create a temp table for the parquet data
p_home_sales_df.createOrReplaceTempView('p_home_sales')

In [24]:
# run the last query with the parquet data
start_time = time.time()

sql = """
SELECT view, ROUND(AVG(price), 2) AS AVG_PRICE
FROM home_sales
GROUP by view
HAVING AVG_PRICE >= 350000
ORDER BY view
"""

spark.sql(sql).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|AVG_PRICE|
+----+---------+
| 100|1026669.5|
|  51|788128.21|
|  52|733780.26|
|  53| 755214.8|
|  54|798684.82|
|  55|771153.32|
|  56| 718176.4|
|  57| 734340.5|
|  58|759764.65|
|  59| 791453.0|
|  60|754939.65|
|  61|746877.59|
|  62|759150.14|
|  63|711614.55|
|  64|767036.67|
|  65|736679.93|
|  66| 712475.0|
|  67|737970.96|
|  68|716785.44|
|  69|750537.94|
+----+---------+
only showing top 20 rows

--- 0.6086246967315674 seconds ---


In [25]:
# Uncache the home_sales temporary table
spark.sql('uncache table home_sales')

DataFrame[]

In [26]:
spark.catalog.isCached('home_sales')

False